# Module 1a: Basic Agent (Without Skills)

In this module, you'll run a **basic Claude agent WITHOUT skills or project context** to see how the agent behaves when it lacks domain expertise.

## Learning Objectives

By the end of this module, you will:
- Understand the Claude Agent SDK basics
- Run an agent with minimal configuration
- Observe challenges when the agent lacks context
- Appreciate why skills and CLAUDE.md matter

## What to Expect

The basic agent will:
- Try to query the database
- **Guess** column names (often incorrectly)
- **Not know** about pre-calculated metrics
- Take more iterations to get correct results

## Step 1: Load Environment and Dependencies

In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# Set up paths
workshop_root = Path("..").resolve()
sys.path.insert(0, str(workshop_root))

# Load environment
load_dotenv(workshop_root / ".env")

# CRITICAL: Ensure Claude Agent SDK uses Bedrock (not Claude API key)
os.environ["CLAUDE_CODE_USE_BEDROCK"] = "1"

print(f"Workshop root: {workshop_root}")
print(f"AWS Region: {os.getenv('AWS_REGION', 'NOT SET')}")
print(f"Database: {os.getenv('ATHENA_DATABASE', 'NOT SET')}")
print(f"Using Bedrock: {os.getenv('CLAUDE_CODE_USE_BEDROCK', 'NOT SET')}")

## Step 2: Understand the Basic Agent

Let's look at what makes the basic agent "basic":

```python
# Basic agent configuration (from agent/basic_agent.py)
options = ClaudeAgentOptions(
    system_prompt=minimal_prompt,     # Very basic instructions
    allowed_tools=["Read", "Write", "Bash"],  # NO Skill tool!
    # setting_sources=[]              # NO skills discovery!
    cwd=project_root,
    max_turns=20
)
```

**Key limitations:**
1. No `Skill` tool - can't load domain expertise
2. No `setting_sources=["project"]` - no automatic skill discovery
3. Minimal system prompt - only basic instructions

In [ ]:
# Let's look at the basic agent code
basic_agent_path = workshop_root / "agent" / "basic_agent.py"

with open(basic_agent_path, 'r') as f:
    content = f.read()

# Find and display the system prompt section
import re
prompt_match = re.search(r'system_prompt = f"""(.+?)"""', content, re.DOTALL)
if prompt_match:
    print("Basic Agent System Prompt:")
    print("=" * 50)
    print(prompt_match.group(1)[:1500] + "...")
else:
    print("Showing basic_agent.py configuration:")
    print("=" * 50)
    print(content[500:2500] + "...")

## Step 3: Run the Basic Agent

Let's ask a simple question and observe how the agent handles it WITHOUT domain skills.

### Query 1: "How many students are currently enrolled?"

In [ ]:
from claude_agent_sdk import ClaudeAgentOptions, tool, create_sdk_mcp_server, ClaudeSDKClient
from tools.athena_tools import AthenaQueryExecutor

# Get configuration
athena_database = os.getenv("ATHENA_DATABASE", "student_analytics")
athena_output = os.getenv("ATHENA_OUTPUT_LOCATION")
aws_region = os.getenv("AWS_REGION", "us-east-1")

# Initialize Athena executor
athena_executor = AthenaQueryExecutor(
    database=athena_database,
    output_location=athena_output,
    results_dir='./results/raw',
    region=aws_region
)

# Create the Athena MCP tool
@tool("execute_athena_query", "Execute SQL queries against Amazon Athena database and download results", {
    "query": str,
    "local_filename": str
})
async def execute_athena_query(args):
    """Execute SQL query on Athena and download results."""
    try:
        query_text = args.get("query", "")
        local_filename = args.get("local_filename", "query_results.csv")
        
        result = athena_executor.execute_and_download(
            query=query_text,
            local_filename=local_filename
        )
        
        response_text = f"""Query completed successfully!
Data scanned: {result.get('data_scanned_bytes', 0) / (1024**2):.2f} MB
Execution time: {result.get('execution_time_ms', 0) / 1000:.2f} seconds
Results downloaded to: {result['local_file']}"""
        
        return {"content": [{"type": "text", "text": response_text}]}
    except Exception as e:
        return {"content": [{"type": "text", "text": f"Error executing query: {str(e)}"}], "isError": True}

# Create MCP server with the Athena tool
athena_server = create_sdk_mcp_server(
    name="athena",
    version="1.0.0",
    tools=[execute_athena_query]
)

# Minimal system prompt - no skills, no detailed context
basic_system_prompt = f"""You are a Student Analytics AI Agent. Help users analyze student data.

Database: {athena_database}
Region: {aws_region}

Use the execute_athena_query tool to run SQL queries. Only SELECT queries are allowed.

Available tables (you may need to explore for exact column names):
- student_enrollment_analytics
- student_academic_performance  
- financial_summary_by_student
- course_performance_analytics
"""

print("Basic agent configured with MCP Athena tool")
print("Notice: NO skills, NO CLAUDE.md, NO column metadata")
print(f"Tool: mcp__athena__execute_athena_query")

In [ ]:
import anyio

async def run_basic_query(user_query: str):
    """Run a query using the basic agent (no skills)"""
    
    # Configure with minimal options - NO Skill tool, but with MCP Athena tool
    options = ClaudeAgentOptions(
        system_prompt=basic_system_prompt,
        allowed_tools=["Read", "Write", "Bash", "mcp__athena__execute_athena_query"],  # NO Skill tool!
        mcp_servers={"athena": athena_server},
        cwd=str(workshop_root),
        max_turns=20
    )
    
    print("=" * 70)
    print("BASIC AGENT (No Skills/Context)")
    print("=" * 70)
    print(f"\nQuery: {user_query}\n")
    print("-" * 70)
    
    tool_calls = []
    
    async with ClaudeSDKClient(options=options) as client:
        await client.query(user_query)
        
        async for message in client.receive_response():
            if hasattr(message, 'subtype'):
                if message.subtype == 'success':
                    duration = getattr(message, 'duration_ms', 0) / 1000
                    cost = getattr(message, 'total_cost_usd', 0)
                    turns = getattr(message, 'num_turns', 0)
                    print(f"\n" + "=" * 70)
                    print(f"Completed: {duration:.1f}s | ${cost:.4f} | {turns} turns")
                    print(f"Tool calls: {len(tool_calls)}")
                    print("=" * 70)
                    continue
            
            if hasattr(message, 'content'):
                for content in (message.content if isinstance(message.content, list) else [message.content]):
                    if hasattr(content, 'text'):
                        print(content.text)
                    elif hasattr(content, 'name'):
                        tool_calls.append(content.name)
                        if content.name == "mcp__athena__execute_athena_query":
                            query_sql = content.input.get('query', '')[:200]
                            print(f"\n[Athena Query: {query_sql}...]")

# Run the query
await run_basic_query("How many students are currently enrolled?")

## Step 4: Observe the Challenges

Watch what happens when we ask a more complex question that requires domain knowledge.

In [ ]:
# Try a more complex query
await run_basic_query("What is the average number of couses taken by students majoring in Computer Science this semester?")

## Step 5: Common Problems Without Skills

You likely observed some of these issues:

### 1. Column Name Guessing
```sql
-- Agent might guess:
SELECT student_name, gpa FROM students

-- But actual columns are:
SELECT student_first_name, student_last_name, student_overall_gpa 
FROM student_academic_performance
```

### 2. Incorrect Filters
Organisations have many conventions and `common sense` in their organisational context. These conventions may not be obvious to AI. 

```sql
-- Agent might guess:
WHERE status = 'enrolled'

-- But correct values are:
WHERE student_status = 'Active' AND enrollment_status = 'Enrolled'
```

### 3. Ambiguity in Human Language
Human languages are ambiguous, which needs to be clearly defined to produce consistent results. For example, human can ask for `top 3 popular lecturers`, which can be interpreted in multiple ways: 
- by student rating
- by number of enrollment
- latest rating, semester rating or ratings across history
- `Lecturers` only or All ranks (Professor, Associate Professor, Assistant Professor, Lecturer, Adjunct)

### 4. More Tool Calls
Without skills, agent may hallucinate. Some agents are smarter, but they would need to do extra exploration to recover from mistakes or failures:
1. Query to explore table structure
2. Fail with wrong column names
3. Try again with corrections
4. Finally succeed (maybe)

## Key Takeaways

### What We Learned

1. **Basic agents work** but are inefficient without context
2. **Column name guessing** leads to errors and retries
3. **Incorrect or inconsistent logic** leads to errors and inconsistencies
4. **More turns = more cost** and slower responses

### The Solution

In the next notebook, we'll add:
- **Skills** - Domain expertise loaded via `Skill` tool
- **CLAUDE.md** - Project context with table schemas
- **Metadata files** - Exact column names and values

These give the agent the knowledge to:
- Use correct column names immediately
- Follow established query patterns
- Clear definition to clear some ambiguity
- Complete tasks in fewer turns

## Next Steps

Continue to:

**[Module 1b: Agent with Skills](./1b-agent-with-skills.ipynb)**

In the next module, you'll:
- Enable skills and CLAUDE.md context
- Run the SAME queries with dramatic improvement
- See how skills transform agent behavior

---

*Workshop: Build Agentic AI Applications with Claude Agent SDK and Amazon Bedrock AgentCore*